In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Concatenate, Dense


from zipfile import ZipFile
import os

# Specify the path to the zip file and extraction path
zip_file_path = "/content/indonesian_tourism_dataset.zip"
extract_path = "/content"

# Extract the zip file
with ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# List the extracted files
extracted_files = os.listdir(extract_path)

# Load datasets
rating = pd.read_csv('/content/tourism_rating.csv')
place = pd.read_csv('/content/tourism_with_id.csv')
user = pd.read_csv('/content/user.csv')

# Merge data based on 'Place_Id'
data = pd.merge(rating, place, on='Place_Id')
data = pd.merge(data, user, on='User_Id')

# Drop unnecessary columns
data.drop(['Time_Minutes', 'Coordinate', 'Location', 'Unnamed: 11', 'Unnamed: 12'], axis=1, inplace=True)

# Select features for content-based filtering
label_encoder = LabelEncoder()
data['Category_encoded'] = label_encoder.fit_transform(data['Category'])

# Normalize the ratings
min_rating = data['Place_Ratings'].min()
max_rating = data['Place_Ratings'].max()
data['Place_Ratings'] = (data['Place_Ratings'] - min_rating) / (max_rating - min_rating)

# Memilih fitur yang akan digunakan
features = ['User_Id', 'Place_Id', 'Place_Ratings', 'Age', 'Category_encoded', 'Lat', 'Long']

# Memisahkan data menjadi set pelatihan dan pengujian
train_data, test_data = train_test_split(data[features], test_size=0.2, random_state=42)

# Membuat model collaborative filtering
# Mendefinisikan input
place_input = Input(shape=(1,), name='place_input')
age_input = Input(shape=(1,), name='age_input')
category_input = Input(shape=(1,), name='category_input')
lat_input = Input(shape=(1,), name='lat_input')
long_input = Input(shape=(1,), name='long_input')
embedding_size = 50

# Mendefinisikan embedding layer untuk place
place_embedding = Embedding(input_dim=len(data['Place_Id'].unique()) + 1, output_dim=embedding_size, input_length=1)(place_input)

# Menggabungkan hasil embedding dari place
place_flatten = Flatten()(place_embedding)

# Menggabungkan dengan input lainnya
merged_inputs = Concatenate()([place_flatten, age_input, category_input, lat_input, long_input])

# Menambahkan layer Dense untuk memproses input yang telah digabungkan
x = Dense(128, activation='relu')(merged_inputs)
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)


# Output layer dengan aktivasi linear untuk rating
output = Dense(1, activation='linear')(x)

# Membuat model
model = Model(inputs=[place_input, age_input, category_input, lat_input, long_input], outputs=output)

# Mengkompilasi model dengan optimizer yang berbeda dan learning rate yang mungkin lebih sesuai
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_absolute_error',metrics=['mae'])

# Melatih model
model.fit([train_data['Place_Id'], train_data['Age'], train_data['Category_encoded'], train_data['Lat'], train_data['Long']],
          train_data['Place_Ratings'], epochs=100, batch_size=64, validation_split=0.2)

Epoch 1/100
100/100 [==============================] - 4s 19ms/step - loss: 0.8296 - mae: 0.8296 - val_loss: 0.3433 - val_mae: 0.3433
Epoch 2/100
100/100 [==============================] - 1s 14ms/step - loss: 0.3470 - mae: 0.3470 - val_loss: 0.5311 - val_mae: 0.5311
Epoch 3/100
100/100 [==============================] - 1s 10ms/step - loss: 0.3666 - mae: 0.3666 - val_loss: 0.3112 - val_mae: 0.3112
Epoch 4/100
100/100 [==============================] - 1s 11ms/step - loss: 0.3333 - mae: 0.3333 - val_loss: 0.3520 - val_mae: 0.3520
Epoch 5/100
100/100 [==============================] - 1s 8ms/step - loss: 0.3210 - mae: 0.3210 - val_loss: 0.3297 - val_mae: 0.3297
Epoch 6/100
100/100 [==============================] - 1s 6ms/step - loss: 0.3217 - mae: 0.3217 - val_loss: 0.3112 - val_mae: 0.3112
Epoch 7/100
100/100 [==============================] - 1s 8ms/step - loss: 0.3093 - mae: 0.3093 - val_loss: 0.3762 - val_mae: 0.3762
Epoch 8/100
100/100 [==============================] - 0s 4ms/ste

In [3]:
model.save('model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [4]:
def recommend(age, category, lat, long):
    category_encoded = label_encoder.transform([category])[0]

    # Membuat array input untuk setiap tempat
    places = data['Place_Id'].unique()
    age_inputs = np.full(len(places), age)
    category_inputs = np.full(len(places), category_encoded)
    lat_inputs = np.full(len(places), lat)
    long_inputs = np.full(len(places), long)

    # Menggunakan model untuk membuat prediksi untuk setiap tempat
    predictions = model.predict([places, age_inputs, category_inputs, lat_inputs, long_inputs]).flatten()

    # Mendapatkan indeks tempat dengan rating tertinggi
    top_indices = np.argsort(predictions)[::-1][:20]

    # Mengembalikan rekomendasi tempat (filter out duplicates)
    recommended_places = data.loc[data['Place_Id'].isin(places[top_indices])]
    unique_recommendations = recommended_places.drop_duplicates(subset=['Place_Id'])

    return unique_recommendations

# Contoh penggunaan fungsi recommend
age = 78  # Ganti dengan umur pengguna
category = 'Tempat Ibadah'  # Ganti dengan kategori yang diinginkan
lat = -6.1753924  # Ganti dengan latitude yang diinginkan
long = 106.8271528  # Ganti dengan longitude yang diinginkan

recommendations = recommend(age, category, lat, long)
recommendations[['Place_Name', 'Category', 'City', 'Price', 'Rating', 'Lat', 'Long']]


14/14 [==============================] - 0s 2ms/step


,Place_Name,Category,City,Price,Rating,Lat,Long
77,Kampung Wisata Kadipaten,Budaya,Yogyakarta,0,4.4,-7.806093,110.358310
93,Kampoeng Tulip,Taman Hiburan,Bandung,15000,3.8,-6.963162,107.661950
116,Taman Prestasi,Taman Hiburan,Surabaya,0,4.6,-7.261472,112.742828
156,Brown Canyon,Cagar Alam,Semarang,0,4.2,-7.056516,110.486253
179,Saung Angklung Mang Udjo,Budaya,Bandung,75000,4.7,-6.897136,107.655847
254,Taman Pandanaran,Taman Hiburan,Semarang,0,4.4,-6.987794,110.417226
260,Roemah Seni Sarasvati,Budaya,Bandung,0,4.4,-6.920417,107.599643
301,Bangsal Pagelaran,Budaya,Yogyakarta,5000,4.6,-7.805284,110.364214
384,Pulau Pelangi,Bahari,Jakarta,900000,4.8,-5.587055,106.588500
540,Pesona Nirwana Waterpark & Cottages,Taman Hiburan,Bandung,50000,4.1,-7.040915,107.526916
